<a href="https://colab.research.google.com/github/MOOwuttichai/python_dataviz2023/blob/main/%E0%B8%87%E0%B8%B2%E0%B8%99%E0%B9%82%E0%B8%9B%E0%B8%A3%E0%B9%80%E0%B8%88%E0%B8%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#งานโปรเจค


สมาชิกกลุ่ม
*   List item
*   List item
*   List item
*   List item
*   List item
*   List item



#หัวข้อที่สนใจ
1. ต้องการทราบว่า พรรคการเมืองใดที่ได้คะแนนสูงสุดของสส.แบบแบ่งเขตในแต่ละจังหวัด
2.  ต้องการทราบว่า พรรคการเมืองใดที่ได้คะแนนสูงสุดของสส.บัญชีรายชื่อในแต่ละจังหวัด
3.  ต้องการทราบว่า ทั่วทั้งประเทศพรรคที่ได้คะแนนเสียงมากที่สุด 5 อันดับแรก มีพรรคอะไรบ้าง
4.  ต้องการทราบว่าใครคือส.ส.เเบบเเบ่งเขตที่มีคะเเนนเสียงมากที่สุดในพรรคการเมืองเเต่ละพรรค
5.  ต้องการทราบ สัดส่วนร้อยละผู้ที่เข้ามาใช้สิทธิ์ในแต่ละภูมิภาค เป็นเท่าใด

#เตรียมข้อมูล

In [6]:
from google.colab import drive # เชื่อมบัญชีกับ google drive
drive.mount('/content/drive') # เช็คว่าเชื่อมติดหรือไม่
import pandas as pd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
result_constituencies_PartyList = pd.read_excel('/content/drive/MyDrive/Dataviz_23data/ectreport2023_มีภาค.xlsx',sheet_name='result_constituencies_PartyList')
result_constituencies_Candidate = pd.read_excel('/content/drive/MyDrive/Dataviz_23data/ectreport2023_มีภาค.xlsx',sheet_name='result_constituencies_Candidate')
result_constituencies_status = pd.read_excel('/content/drive/MyDrive/Dataviz_23data/ectreport2023_มีภาค.xlsx',sheet_name='result_constituencies_status')
info_province = pd.read_excel('/content/drive/MyDrive/Dataviz_23data/ectreport2023_มีภาค.xlsx',sheet_name='info_province')
info_party_overview = pd.read_excel('/content/drive/MyDrive/Dataviz_23data/ectreport2023_มีภาค.xlsx',sheet_name='info_party_overview')
info_constituency = pd.read_excel('/content/drive/MyDrive/Dataviz_23data/ectreport2023_มีภาค.xlsx',sheet_name='info_constituency')
Candidate_Constituency = pd.read_excel('/content/drive/MyDrive/Dataviz_23data/ectreport2023_มีภาค.xlsx',sheet_name='Candidate_Constituency')
Candidate_PartyList = pd.read_excel('/content/drive/MyDrive/Dataviz_23data/ectreport2023_มีภาค.xlsx',sheet_name='Candidate_PartyList')
Candidate_PM = pd.read_excel('/content/drive/MyDrive/Dataviz_23data/ectreport2023_มีภาค.xlsx',sheet_name='Candidate_PM')
region = pd.read_excel('/content/drive/MyDrive/Dataviz_23data/ectreport2023_มีภาค.xlsx',sheet_name='region')

เครียส์MISSing

In [8]:
info_party_overview[info_party_overview['party_no'].isnull()]

,id,party_no,name,abbr,color,logo_url
9,711,NaN,ประชาสามัคคี,ปส.,#ecb55e,https://cdn.ectreport.com/images/logos/ประชาสา...
11,713,NaN,พลังบูรพา,พบ.,#4696d6,https://cdn.ectreport.com/images/logos/พลังบูร...
17,720,NaN,รักษ์ธรรม,ร.ธ.,#dbdc9c,https://cdn.ectreport.com/images/logos/รักษ์ธร...


In [9]:
info_party_overview['party_no']=info_party_overview['party_no'].fillna(99)

In [10]:
info_party_overview[info_party_overview['party_no'].isnull()]

,id,party_no,name,abbr,color,logo_url


In [11]:
info_constituency[info_constituency['zone'].isnull()]

,cons_id,cons_no,prov_id,registered_vote,total_vote_stations,zone
804,RNG_1,1,RNG,140024,209,NaN
817,SBR_1,1,SBR,168055,213,NaN
839,SKM_1,1,SKM,156558,257,NaN
1037,TRT_1,1,TRT,175112,314,NaN


In [12]:
import numpy as np
# แทนค่าว่าง
info_constituency['zone'] = np.where(info_constituency['prov_id'] == 'RNG', 'อำเภอเมืองระนอง', info_constituency['zone'])
info_constituency['zone'] = np.where(info_constituency['prov_id'] == 'SBR', 'อำเภอเมืองสิงห์บุรี', info_constituency['zone'])
info_constituency['zone'] = np.where(info_constituency['prov_id'] == 'SKM', 'อำเภอเมืองสมุทรสงคราม', info_constituency['zone'])
info_constituency['zone'] = np.where(info_constituency['prov_id'] == 'TRT', 'อำเภอเมืองตราด', info_constituency['zone'])

In [13]:
info_constituency[info_constituency['zone'].isnull()]

,cons_id,cons_no,prov_id,registered_vote,total_vote_stations,zone


1.แต่ละจังหวัดเลือกพรรคอะไรมากที่สุด

In [14]:
table_merge = result_constituencies_Candidate.merge(Candidate_Constituency, how ='left', left_on='mp_app_id',right_on='mp_app_id')
print(table_merge.shape)
table_merge.head()

(4781, 10)


,cons_id,mp_app_id,mp_app_rank,mp_app_vote,mp_app_vote_percent,party_id,mp_app_name,mp_app_no,mp_app_party_id,image_url
0,ACR_1,ACR_1_1,9,459,0.43477,706,นางสาวปิยะมาศ บัวแก้ว,1,706,https://cdn.ectreport.com/images/mp_candidates...
1,ACR_1,ACR_1_10,1,38275,36.25419,709,นางสุขสมรวย วันทนียกุล,10,709,https://cdn.ectreport.com/images/mp_candidates...
2,ACR_1,ACR_1_11,13,126,0.11935,712,นายการันต์ ลุนพันธ์,11,712,https://cdn.ectreport.com/images/mp_candidates...
3,ACR_1,ACR_1_12,12,127,0.12029,742,นายไสว ผลาวงศ์,12,742,https://cdn.ectreport.com/images/mp_candidates...
4,ACR_1,ACR_1_13,11,212,0.20081,761,นายสงบ โสภิพันธ์,13,761,https://cdn.ectreport.com/images/mp_candidates...


In [15]:
table_merge1 = table_merge.merge(info_party_overview, how ='left', left_on='party_id',right_on='id')
print(table_merge1.shape)
table_merge1.head()

(4781, 16)


,cons_id,mp_app_id,mp_app_rank,mp_app_vote,mp_app_vote_percent,party_id,mp_app_name,mp_app_no,mp_app_party_id,image_url,id,party_no,name,abbr,color,logo_url
0,ACR_1,ACR_1_1,9,459,0.43477,706,นางสาวปิยะมาศ บัวแก้ว,1,706,https://cdn.ectreport.com/images/mp_candidates...,706,14.0,ชาติพัฒนากล้า,ชพก.,#233e8a,https://cdn.ectreport.com/images/logos/ชาติพัฒ...
1,ACR_1,ACR_1_10,1,38275,36.25419,709,นางสุขสมรวย วันทนียกุล,10,709,https://cdn.ectreport.com/images/mp_candidates...,709,7.0,ภูมิใจไทย,ภท.,#0c149c,https://cdn.ectreport.com/images/logos/ภูมิใจไ...
2,ACR_1,ACR_1_11,13,126,0.11935,712,นายการันต์ ลุนพันธ์,11,712,https://cdn.ectreport.com/images/mp_candidates...,712,2.0,ประชาธิปไตยใหม่,ปธม.,#eb4138,https://cdn.ectreport.com/images/logos/ประชาธิ...
3,ACR_1,ACR_1_12,12,127,0.12029,742,นายไสว ผลาวงศ์,12,742,https://cdn.ectreport.com/images/mp_candidates...,742,61.0,คลองไทย,คล.ท.,#0eafeb,https://cdn.ectreport.com/images/logos/คลองไทย...
4,ACR_1,ACR_1_13,11,212,0.20081,761,นายสงบ โสภิพันธ์,13,761,https://cdn.ectreport.com/images/mp_candidates...,761,5.0,พลังสังคมใหม่,พ.ส.ม.,#eabcc3,https://cdn.ectreport.com/images/logos/พลังสัง...


In [16]:
table_merge2 = table_merge1.merge(result_constituencies_status, how ='left', left_on='cons_id',right_on='cons_id')
print(table_merge2.shape)
table_merge2.head()

(4781, 22)


,cons_id,mp_app_id,mp_app_rank,mp_app_vote,mp_app_vote_percent,party_id,mp_app_name,mp_app_no,mp_app_party_id,image_url,...,name,abbr,color,logo_url,prov_id,turn_out,percent_turn_out,counted_vote_stations,percent_count,pause_report
0,ACR_1,ACR_1_1,9,459,0.43477,706,นางสาวปิยะมาศ บัวแก้ว,1,706,https://cdn.ectreport.com/images/mp_candidates...,...,ชาติพัฒนากล้า,ชพก.,#233e8a,https://cdn.ectreport.com/images/logos/ชาติพัฒ...,ACR,105574,72.60136,310,100.0,False
1,ACR_1,ACR_1_10,1,38275,36.25419,709,นางสุขสมรวย วันทนียกุล,10,709,https://cdn.ectreport.com/images/mp_candidates...,...,ภูมิใจไทย,ภท.,#0c149c,https://cdn.ectreport.com/images/logos/ภูมิใจไ...,ACR,105574,72.60136,310,100.0,False
2,ACR_1,ACR_1_11,13,126,0.11935,712,นายการันต์ ลุนพันธ์,11,712,https://cdn.ectreport.com/images/mp_candidates...,...,ประชาธิปไตยใหม่,ปธม.,#eb4138,https://cdn.ectreport.com/images/logos/ประชาธิ...,ACR,105574,72.60136,310,100.0,False
3,ACR_1,ACR_1_12,12,127,0.12029,742,นายไสว ผลาวงศ์,12,742,https://cdn.ectreport.com/images/mp_candidates...,...,คลองไทย,คล.ท.,#0eafeb,https://cdn.ectreport.com/images/logos/คลองไทย...,ACR,105574,72.60136,310,100.0,False
4,ACR_1,ACR_1_13,11,212,0.20081,761,นายสงบ โสภิพันธ์,13,761,https://cdn.ectreport.com/images/mp_candidates...,...,พลังสังคมใหม่,พ.ส.ม.,#eabcc3,https://cdn.ectreport.com/images/logos/พลังสัง...,ACR,105574,72.60136,310,100.0,False


In [17]:
table_merge3 = table_merge2.merge(info_province, how ='left', left_on='prov_id',right_on='prov_id')
print(table_merge3.shape)
table_merge3.head()

(4781, 29)


,cons_id,mp_app_id,mp_app_rank,mp_app_vote,mp_app_vote_percent,party_id,mp_app_name,mp_app_no,mp_app_party_id,image_url,...,counted_vote_stations,percent_count,pause_report,province_id,province,eng,abbre_thai,total_registered_vote,total_vote_stations,region_id
0,ACR_1,ACR_1_1,9,459,0.43477,706,นางสาวปิยะมาศ บัวแก้ว,1,706,https://cdn.ectreport.com/images/mp_candidates...,...,310,100.0,False,37,อำนาจเจริญ,AMNAT CHAROEN,อจ,301354,641,R02
1,ACR_1,ACR_1_10,1,38275,36.25419,709,นางสุขสมรวย วันทนียกุล,10,709,https://cdn.ectreport.com/images/mp_candidates...,...,310,100.0,False,37,อำนาจเจริญ,AMNAT CHAROEN,อจ,301354,641,R02
2,ACR_1,ACR_1_11,13,126,0.11935,712,นายการันต์ ลุนพันธ์,11,712,https://cdn.ectreport.com/images/mp_candidates...,...,310,100.0,False,37,อำนาจเจริญ,AMNAT CHAROEN,อจ,301354,641,R02
3,ACR_1,ACR_1_12,12,127,0.12029,742,นายไสว ผลาวงศ์,12,742,https://cdn.ectreport.com/images/mp_candidates...,...,310,100.0,False,37,อำนาจเจริญ,AMNAT CHAROEN,อจ,301354,641,R02
4,ACR_1,ACR_1_13,11,212,0.20081,761,นายสงบ โสภิพันธ์,13,761,https://cdn.ectreport.com/images/mp_candidates...,...,310,100.0,False,37,อำนาจเจริญ,AMNAT CHAROEN,อจ,301354,641,R02


In [18]:
table_merge3 = table_merge2.merge(info_province, how ='left', left_on='prov_id',right_on='prov_id')
print(table_merge3.shape)
table_merge3.head()

(4781, 29)


,cons_id,mp_app_id,mp_app_rank,mp_app_vote,mp_app_vote_percent,party_id,mp_app_name,mp_app_no,mp_app_party_id,image_url,...,counted_vote_stations,percent_count,pause_report,province_id,province,eng,abbre_thai,total_registered_vote,total_vote_stations,region_id
0,ACR_1,ACR_1_1,9,459,0.43477,706,นางสาวปิยะมาศ บัวแก้ว,1,706,https://cdn.ectreport.com/images/mp_candidates...,...,310,100.0,False,37,อำนาจเจริญ,AMNAT CHAROEN,อจ,301354,641,R02
1,ACR_1,ACR_1_10,1,38275,36.25419,709,นางสุขสมรวย วันทนียกุล,10,709,https://cdn.ectreport.com/images/mp_candidates...,...,310,100.0,False,37,อำนาจเจริญ,AMNAT CHAROEN,อจ,301354,641,R02
2,ACR_1,ACR_1_11,13,126,0.11935,712,นายการันต์ ลุนพันธ์,11,712,https://cdn.ectreport.com/images/mp_candidates...,...,310,100.0,False,37,อำนาจเจริญ,AMNAT CHAROEN,อจ,301354,641,R02
3,ACR_1,ACR_1_12,12,127,0.12029,742,นายไสว ผลาวงศ์,12,742,https://cdn.ectreport.com/images/mp_candidates...,...,310,100.0,False,37,อำนาจเจริญ,AMNAT CHAROEN,อจ,301354,641,R02
4,ACR_1,ACR_1_13,11,212,0.20081,761,นายสงบ โสภิพันธ์,13,761,https://cdn.ectreport.com/images/mp_candidates...,...,310,100.0,False,37,อำนาจเจริญ,AMNAT CHAROEN,อจ,301354,641,R02


In [19]:
table_merge4 = table_merge3.merge(region, how ='left', left_on='region_id',right_on='region_id')
print(table_merge3.shape)
table_merge4.head()

(4781, 29)


,cons_id,mp_app_id,mp_app_rank,mp_app_vote,mp_app_vote_percent,party_id,mp_app_name,mp_app_no,mp_app_party_id,image_url,...,percent_count,pause_report,province_id,province,eng,abbre_thai,total_registered_vote,total_vote_stations,region_id,region_name
0,ACR_1,ACR_1_1,9,459,0.43477,706,นางสาวปิยะมาศ บัวแก้ว,1,706,https://cdn.ectreport.com/images/mp_candidates...,...,100.0,False,37,อำนาจเจริญ,AMNAT CHAROEN,อจ,301354,641,R02,ภาคตะวันออกเฉียงเหนือ
1,ACR_1,ACR_1_10,1,38275,36.25419,709,นางสุขสมรวย วันทนียกุล,10,709,https://cdn.ectreport.com/images/mp_candidates...,...,100.0,False,37,อำนาจเจริญ,AMNAT CHAROEN,อจ,301354,641,R02,ภาคตะวันออกเฉียงเหนือ
2,ACR_1,ACR_1_11,13,126,0.11935,712,นายการันต์ ลุนพันธ์,11,712,https://cdn.ectreport.com/images/mp_candidates...,...,100.0,False,37,อำนาจเจริญ,AMNAT CHAROEN,อจ,301354,641,R02,ภาคตะวันออกเฉียงเหนือ
3,ACR_1,ACR_1_12,12,127,0.12029,742,นายไสว ผลาวงศ์,12,742,https://cdn.ectreport.com/images/mp_candidates...,...,100.0,False,37,อำนาจเจริญ,AMNAT CHAROEN,อจ,301354,641,R02,ภาคตะวันออกเฉียงเหนือ
4,ACR_1,ACR_1_13,11,212,0.20081,761,นายสงบ โสภิพันธ์,13,761,https://cdn.ectreport.com/images/mp_candidates...,...,100.0,False,37,อำนาจเจริญ,AMNAT CHAROEN,อจ,301354,641,R02,ภาคตะวันออกเฉียงเหนือ


#เรียงคะเเนนเสียงจากรายชื่อว่าที่ ส.ส. จากมากไปน้อย โดยต้องการรู้ "ชื่อจังหวัด พรรคที่สังกัด ชื่อว่าที่ส.ส.เเละคะเเนนเสียง"

In [20]:
find_ma = result_constituencies_Candidate.sort_values(by='mp_app_vote',ascending=False)
find_ma1 = find_ma.merge(info_constituency,how='left', on='cons_id')
find_ma2 = find_ma1.merge(info_province,how='left', on='prov_id')
find_ma3 = find_ma2.merge(info_party_overview,how='left', left_on='party_id',right_on='id')
find_ma4 = find_ma3.merge(Candidate_Constituency,how='left', on='mp_app_id')

In [21]:
find_mashow =find_ma4[['province','name','mp_app_name']]
find_mashow['mp_app_vote_'] = find_ma4[['mp_app_vote']]
find_mashow

<ipython-input-21-9fd55a1fecaa>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  find_mashow['mp_app_vote_'] = find_ma4[['mp_app_vote']]


,province,name,mp_app_name,mp_app_vote_
0,ตรัง,พลังประชารัฐ,นายทวี สุระบาล,63185
1,ตรัง,พลังประชารัฐ,นายทวี สุระบาล,63185
2,ตรัง,พลังประชารัฐ,นายทวี สุระบาล,63185
3,อุบลราชธานี,เพื่อไทรวมพลัง,นายสมศักดิ์ บุญประชม,63127
4,อุบลราชธานี,เพื่อไทรวมพลัง,นายสมศักดิ์ บุญประชม,63127
...,...,...,...,...
13503,สงขลา,คลองไทย,นางลักขณา บุญรักษ์,0
13504,นนทบุรี,พลังสังคม,นายรัฐศิลป์ นลินธนาพัฒน์,0
13505,นนทบุรี,พลังสังคม,นายรัฐศิลป์ นลินธนาพัฒน์,0
13506,สงขลา,คลองไทย,นายมีชัย เทศนอก,0


In [22]:
use_V0=find_mashow[['name','mp_app_name','mp_app_vote_']]
prov_vote=find_mashow.groupby('province')[['mp_app_vote_']].max().reset_index()
use_V1 = prov_vote.merge(use_V0,how='left', on='mp_app_vote_')
bool_use = use_V1.duplicated(subset='province')
use_V1 = use_V1[~bool_use]
# displaying data
use_V1.info()
use_V1

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77 entries, 0 to 232
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   province      77 non-null     object
 1   mp_app_vote_  77 non-null     int64 
 2   name          77 non-null     object
 3   mp_app_name   77 non-null     object
dtypes: int64(1), object(3)
memory usage: 3.0+ KB


,province,mp_app_vote_,name,mp_app_name
0,กระบี่,51845,ภูมิใจไทย,นายสฤษฏ์พงษ์ เกี่ยวข้อง
4,กรุงเทพมหานคร,50132,ก้าวไกล,นายศุภณัฐ มีนชัยนันท์
7,กาญจนบุรี,44813,ภูมิใจไทย,นายยศวัฒน์ มาไพศาลสิน
9,กาฬสินธุ์,56559,เพื่อไทย,นายวิรัช พิมพะนิตย์
12,กำแพงเพชร,41143,พลังประชารัฐ,นายเพชรภูมิ อาภรณ์รัตน์
...,...,...,...,...
221,เพชรบุรี,40235,รวมไทยสร้างชาติ,นางธิวัลรัตน์ อังกินันทน์
223,เพชรบูรณ์,47118,พลังประชารัฐ,นายอัคร ทองใจสด
225,เลย,50421,เพื่อไทย,นายศรัณย์ ทิมสุวรรณ
230,แพร่,41565,เพื่อไทย,นายทศพร เสรีรักษ์


#ข้อ 2

พรรคก้าวไกล

In [23]:
find_proforcon1 = result_constituencies_Candidate.merge(info_party_overview,how='left', left_on='party_id',right_on='id')
find_proforcon = find_proforcon1.merge(Candidate_Constituency,how='left', on='mp_app_id')

In [24]:
fastfor_w = find_proforcon[find_proforcon['name']=='ก้าวไกล']
TOP_fast = fastfor_w[['mp_app_name','mp_app_vote','mp_app_vote_percent']]
TOP_fast_soft = TOP_fast.sort_values(by='mp_app_vote_percent', ascending=False)
TOP_fast_soft

,mp_app_name,mp_app_vote,mp_app_vote_percent
2605,นายปิยวัฒน์ สิริพันธ์พงศ์,10700,279.88491
960,นางสาวพุธิตา ชัยอนันต์,62009,54.01387
3910,นางสาวนิตยา มีศรี,60282,50.24547
386,นายณัฐชา บุญไชยอินสวัสดิ์,50035,49.46907
3900,นายวุฒินันท์ บุญชู,49043,48.17110
...,...,...,...
3182,นายโมฮำมัดรอฮมัด มามุ,4558,5.43337
2730,นายวรนารถ นันทกิจ,4673,4.24270
2678,นายมะซอบือรี สามะ,3622,4.23254
3052,นายสุนทร คมคาย,4218,4.15149


In [25]:
def math_pa(x):
  y=(100*x)/1220778
  return y

In [26]:
final_TOP1 = TOP_fast_soft.replace(to_replace=279.88491,
           value=math_pa(279.88491)).sort_values(by='mp_app_vote_percent', ascending=False)

In [27]:
final_TOP1

,mp_app_name,mp_app_vote,mp_app_vote_percent
960,นางสาวพุธิตา ชัยอนันต์,62009,54.013870
3910,นางสาวนิตยา มีศรี,60282,50.245470
386,นายณัฐชา บุญไชยอินสวัสดิ์,50035,49.469070
3900,นายวุฒินันท์ บุญชู,49043,48.171100
449,นายธัญธร ธนินวัฒนาธร,49770,47.516760
...,...,...,...
2730,นายวรนารถ นันทกิจ,4673,4.242700
2678,นายมะซอบือรี สามะ,3622,4.232540
3052,นายสุนทร คมคาย,4218,4.151490
2688,นายโยฮัน สาตออุมา,3312,3.457090


พรรคเพื่อไทย

In [28]:
forthai_w = find_proforcon[find_proforcon['name']=='เพื่อไทย']
TOP_thai = forthai_w[['mp_app_name','mp_app_vote','mp_app_vote_percent']]
TOP_thai_soft = TOP_thai.sort_values(by='mp_app_vote_percent', ascending=False)
TOP_thai_soft

,mp_app_name,mp_app_vote,mp_app_vote_percent
2593,นายถาวรวัฒน์ คงแก้ว,5596,146.37719
3335,นางสาวจิราพร สินธุไพร,61288,69.34758
1077,นายเชิงชาย ชาลีรินทร์,61414,63.43832
1579,นายวิรัช พิมพะนิตย์,56559,59.80965
2171,นายอาทิตย์ หวังศุภกิจโกศล,60320,58.86831
...,...,...,...
3183,นายอาหมัดบูรฮัน ติพอง,641,0.76410
2829,นายชัยจรงค์ ฤทธิสุนทร,823,0.72252
3168,นายรุสดี แวบือซา,499,0.71840
3493,ร้อยตำรวจตรีสมศักดิ์ แสงประดับ,720,0.70016


In [29]:
final_TOP2 = TOP_thai_soft.replace(to_replace=146.37719,
           value=math_pa(146.37719)).sort_values(by='mp_app_vote_percent', ascending=False)

In [30]:
final_TOP2

,mp_app_name,mp_app_vote,mp_app_vote_percent
3335,นางสาวจิราพร สินธุไพร,61288,69.34758
1077,นายเชิงชาย ชาลีรินทร์,61414,63.43832
1579,นายวิรัช พิมพะนิตย์,56559,59.80965
2171,นายอาทิตย์ หวังศุภกิจโกศล,60320,58.86831
1407,นายภาควัต ศรีสุรพล,50423,57.89625
...,...,...,...
2829,นายชัยจรงค์ ฤทธิสุนทร,823,0.72252
3168,นายรุสดี แวบือซา,499,0.71840
3493,ร้อยตำรวจตรีสมศักดิ์ แสงประดับ,720,0.70016
2670,นายปักสู แวนามะ,517,0.60415


พรรคภูมิใจไทย

In [31]:
poothai_w = find_proforcon[find_proforcon['name']=='ภูมิใจไทย']
TOP_poo = poothai_w[['mp_app_name','mp_app_vote','mp_app_vote_percent']]
TOP_poo_soft = TOP_poo.sort_values(by='mp_app_vote_percent', ascending=False)
TOP_poo_soft

,mp_app_name,mp_app_vote,mp_app_vote_percent
2606,นายสมศักดิ์ แสงอารยะกุล,23260,608.42270
4658,นายเจเศรษฐ์ ไทยเศรษฐ์,49464,53.66372
687,นายโสภณ ซารัมย์,47746,53.10126
647,นายไชยชนก ชิดชอบ,46729,52.87103
4242,นายวรศิษฎ์ เลียงประสิทธิ์,55612,52.71979
...,...,...,...
879,นางสาวเสาวลักษณ์ น้อยคำเมือง,165,0.13730
1581,นางบุณยะพร พิรชัช,123,0.13550
3872,นายธนภาค ลี่สิน,128,0.12884
1079,นายไตรภูมิ แซ่อึง,115,0.11879


In [32]:
final_TOP3 = TOP_poo_soft.replace(to_replace=608.42270,
           value=math_pa(608.42270)).sort_values(by='mp_app_vote_percent', ascending=False)
final_TOP3

,mp_app_name,mp_app_vote,mp_app_vote_percent
4658,นายเจเศรษฐ์ ไทยเศรษฐ์,49464,53.663720
687,นายโสภณ ซารัมย์,47746,53.101260
647,นายไชยชนก ชิดชอบ,46729,52.871030
4242,นายวรศิษฎ์ เลียงประสิทธิ์,55612,52.719790
1296,นายสฤษฏ์พงษ์ เกี่ยวข้อง,51845,52.130680
...,...,...,...
1581,นางบุณยะพร พิรชัช,123,0.135500
3872,นายธนภาค ลี่สิน,128,0.128840
1079,นายไตรภูมิ แซ่อึง,115,0.118790
2386,นางสาวอมรรัตน์ รุ่งเรือง,95,0.088910


พรรค พลังประชารัฐ

In [33]:
power_w = find_proforcon[find_proforcon['name']=='พลังประชารัฐ']
TOP_power_w = power_w[['mp_app_name','mp_app_vote','mp_app_vote_percent']]
TOP_power_w = TOP_power_w.sort_values(by='mp_app_vote_percent', ascending=False)
TOP_power_w

,mp_app_name,mp_app_vote,mp_app_vote_percent
2602,นายสุธรรม จริตงาม,27331,714.90976
4301,นายทวี สุระบาล,63185,60.29103
3206,นายธรรมนัส พรหมเผ่า,53963,53.06201
2998,นายอัคร ทองใจสด,47118,50.26349
2978,นายวรโชติ สุคนธ์ขจร,41039,47.41104
...,...,...,...
2164,นายณัฐพล ชวนกระโทก,182,0.15867
2726,พลอากาศโทชนนนาถ เทพลิบ,132,0.11617
3054,นายสุรเดช สิทธิเดชกุลถาวร,57,0.05610
1044,นายธนบดี คุ้มชนะ,0,0.00000


In [34]:
final_TOP4 = TOP_power_w.replace(to_replace=714.90976,
           value=math_pa(714.90976)).sort_values(by='mp_app_vote_percent', ascending=False)
final_TOP4

,mp_app_name,mp_app_vote,mp_app_vote_percent
4301,นายทวี สุระบาล,63185,60.291030
3206,นายธรรมนัส พรหมเผ่า,53963,53.062010
2998,นายอัคร ทองใจสด,47118,50.263490
2978,นายวรโชติ สุคนธ์ขจร,41039,47.411040
3605,นางสาวตรีนุช เทียนทอง,47173,45.797250
...,...,...,...
2726,พลอากาศโทชนนนาถ เทพลิบ,132,0.116170
2602,นายสุธรรม จริตงาม,27331,0.058562
3054,นายสุรเดช สิทธิเดชกุลถาวร,57,0.056100
1044,นายธนบดี คุ้มชนะ,0,0.000000


พรรค รวมไทยสร้าง

In [35]:
how_are = find_proforcon[find_proforcon['name']=='รวมไทยสร้างชาติ']
TOP_how_are = how_are[['mp_app_name','mp_app_vote','mp_app_vote_percent']]
TOP_how_are= TOP_how_are.sort_values(by='mp_app_vote_percent', ascending=False)
TOP_how_are

,mp_app_name,mp_app_vote,mp_app_vote_percent
2594,นายฉัตรชัย ธนาวุฒิ,7731,202.22338
3650,นางสาววชิราภรณ์ กาญจนะ,46456,50.16684
3226,นางสาวกุลวลี นพอมรบดี,52941,48.33029
3262,นายอัครเดช วงษ์พิทักษ์โรจน์,54815,47.13525
2536,นางสาวพิมพ์ภัทรา วิชัยกุล,44233,46.09429
...,...,...,...
4426,นายจำรูญศักดิ์ จันทรมัย,0,0.00000
4053,นายธีระทัศน์ เตียวเจริญโสภา,0,0.00000
4161,นายอำไพ เพ็งแจ่ม,0,0.00000
4388,นายเชษฐา ไชยสัตย์,0,0.00000


In [36]:
final_TOP5 = TOP_how_are.replace(to_replace=202.22338,
           value=math_pa(202.22338)).sort_values(by='mp_app_vote_percent', ascending=False)
final_TOP5

,mp_app_name,mp_app_vote,mp_app_vote_percent
3650,นางสาววชิราภรณ์ กาญจนะ,46456,50.16684
3226,นางสาวกุลวลี นพอมรบดี,52941,48.33029
3262,นายอัครเดช วงษ์พิทักษ์โรจน์,54815,47.13525
2536,นางสาวพิมพ์ภัทรา วิชัยกุล,44233,46.09429
3017,นายชนาธิป ศุภศิริ,29063,44.01617
...,...,...,...
4426,นายจำรูญศักดิ์ จันทรมัย,0,0.00000
4053,นายธีระทัศน์ เตียวเจริญโสภา,0,0.00000
4161,นายอำไพ เพ็งแจ่ม,0,0.00000
4388,นายเชษฐา ไชยสัตย์,0,0.00000


#ข้อ 3

In [37]:
bool_use = table_merge4.duplicated(subset='turn_out')
table_merge4 = table_merge4[~bool_use]
# displaying data
use_V1.info()
use_V1

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77 entries, 0 to 232
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   province      77 non-null     object
 1   mp_app_vote_  77 non-null     int64 
 2   name          77 non-null     object
 3   mp_app_name   77 non-null     object
dtypes: int64(1), object(3)
memory usage: 3.0+ KB


,province,mp_app_vote_,name,mp_app_name
0,กระบี่,51845,ภูมิใจไทย,นายสฤษฏ์พงษ์ เกี่ยวข้อง
4,กรุงเทพมหานคร,50132,ก้าวไกล,นายศุภณัฐ มีนชัยนันท์
7,กาญจนบุรี,44813,ภูมิใจไทย,นายยศวัฒน์ มาไพศาลสิน
9,กาฬสินธุ์,56559,เพื่อไทย,นายวิรัช พิมพะนิตย์
12,กำแพงเพชร,41143,พลังประชารัฐ,นายเพชรภูมิ อาภรณ์รัตน์
...,...,...,...,...
221,เพชรบุรี,40235,รวมไทยสร้างชาติ,นางธิวัลรัตน์ อังกินันทน์
223,เพชรบูรณ์,47118,พลังประชารัฐ,นายอัคร ทองใจสด
225,เลย,50421,เพื่อไทย,นายศรัณย์ ทิมสุวรรณ
230,แพร่,41565,เพื่อไทย,นายทศพร เสรีรักษ์


#ข้อ 4
ใช้ ```table_merge4```  เเละไปทำต่อที่ looker

## ลอง

In [38]:
#mpvp =find_proforconAB[find_proforconAB['mp_app_vote_percent'] > 100]

In [39]:
#mpvp

In [40]:
#list(mpvp['mp_app_vote_percent'])

In [41]:
#l = []
#for i in list(mpvp['mp_app_vote_percent']):
 # b = math_pa(i)
  #l.append(b)
#mpvp['mp_app_vote_percent'] = l

In [42]:
LL=table_merge4[['province','region_name','turn_out']]
bool_use = LL.duplicated(subset='province')
LL = LL[~bool_use]
LL

,province,region_name,turn_out
0,อำนาจเจริญ,ภาคตะวันออกเฉียงเหนือ,105574
25,อ่างทอง,ภาคกลาง,89756
42,พระนครศรีอยุธยา,ภาคกลาง,110317
94,กรุงเทพมหานคร,ภาคกลาง,94407
592,บึงกาฬ,ภาคตะวันออกเฉียงเหนือ,75682
...,...,...,...
4494,อุดรธานี,ภาคตะวันออกเฉียงเหนือ,90592
4655,อุทัยธานี,ภาคกลาง,92174
4671,อุตรดิตถ์,ภาคเหนือ,90413
4701,ยะลา,ภาคใต้,99228


In [43]:
re_ture = LL.groupby('region_name').sum('turn_out')
re_ture.sum()

turn_out    7840532
dtype: int64

#ข้อ 5



---



In [44]:
RR=table_merge4[['percent_turn_out','turn_out','province','region_name']]

In [45]:
KK=result_constituencies_status.merge(info_province, how='left',on='prov_id')
JJ = KK.merge(region, how='left',on='region_id')



---



In [46]:
AA = JJ[['region_id','region_name']]

In [47]:
LL = JJ[['region_id','turn_out']]
DD=LL.groupby('region_id').sum().reset_index()

In [48]:
RR = info_province[['total_registered_vote','region_id']]
MM=RR.groupby('region_id').sum().reset_index()

In [49]:
MM['total_registered_vote'].sum()

52238594



---



In [50]:
BBB = DD.merge(MM, how='left',on='region_id')

In [51]:
AAA=BBB.merge(AA, how='left',on='region_id')
bool_use = AAA.duplicated(subset='region_id')
AAA = AAA[~bool_use]
# displaying data
AAA.info()
AAA

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 0 to 371
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   region_id              6 non-null      object
 1   turn_out               6 non-null      int64 
 2   total_registered_vote  6 non-null      int64 
 3   region_name            6 non-null      object
dtypes: int64(2), object(2)
memory usage: 240.0+ bytes


,region_id,turn_out,total_registered_vote,region_name
0,R01,3878723,4912950,ภาคเหนือ
37,R02,12438019,17453038,ภาคตะวันออกเฉียงเหนือ
170,R03,12446904,16285167,ภาคกลาง
292,R04,5525020,7200448,ภาคใต้
352,R05,2012921,2552747,ภาคตะวันตก
371,R06,2983165,3834244,ภาคตะวันออก


#เช็กcolumns เเละเก็บข้อมูลเป็น csv

In [52]:
AAA.to_csv('NNN.csv', index=False)

In [53]:
final_TOP4.to_csv('ข้อมูลพลังประชารัฐ.csv', index=False)

In [54]:
final_TOP3.to_csv('ข้อมูลภูมิใจไทย.csv', index=False)

In [55]:
final_TOP2.to_csv('ข้อมูลเพื่อไทย.csv', index=False)

In [58]:
table_merge3.columns

Index(['cons_id', 'mp_app_id', 'mp_app_rank', 'mp_app_vote',
       'mp_app_vote_percent', 'party_id', 'mp_app_name', 'mp_app_no',
       'mp_app_party_id', 'image_url', 'id', 'party_no', 'name', 'abbr',
       'color', 'logo_url', 'prov_id', 'turn_out', 'percent_turn_out',
       'counted_vote_stations', 'percent_count', 'pause_report', 'province_id',
       'province', 'eng', 'abbre_thai', 'total_registered_vote',
       'total_vote_stations', 'region_id'],
      dtype='object')

In [59]:
table_merge4.to_csv('table_merge_result_useforproject.csv', index=False)

In [60]:
use_V1.to_csv('use_V1.csv', index=False)

In [64]:
BBB.to_csv('นว2.csv', index=False)

In [61]:
all1 = pd.concat([final_TOP1.head(5),final_TOP2.head(5),final_TOP3.head(5),final_TOP4.head(5),final_TOP5.head(5)])

In [62]:
#all1

In [63]:
final_TOP1.to_csv('ข้อมูลก้าวไกล.csv', index=False)